## Table of Contents
0. Initial Setup
1. Utility Functions
2. Network Loading and Preprocessing
   - 2.1 Load Networks
3. Community Detection Analysis
   - 3.1 Apply Infomap, Louvain and Greedy Algorithms
4. Comparison with Ground Truth
   - 4.1 Compute Jaccard Index, NMI, and NVI
   - 4.2 Plots
     - 4.2.1 Number of Detected Communities
     - 4.2.2 Modularity vs prr
     - 4.2.3 Jaccard Index vs prr
     - 4.2.4 Normalized Mutual Information (NMI) vs prr
     - 4.2.5 Variation of Information (NVI) vs prr
5. Network Visualization
   - 5.1 Color-coded Community Visualization for prr = 0.02, 0.16, 1.00
6. Discussion
   - 6.1 Limitations of Modularity
     - Q1: What are the limitations of modularity as a metric for evaluating community detection results?
     - Q2: Can we state that a network with Q = 0.4 has community structure?
   - 6.2 Observations on Differences between Algorithms
     - Q1: What is the difference between Louvain, Greedy and Infomap Algorithms?
     - Q2: Are the modularity values obtained equal? Why?



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
from collections import Counter
from cdlib import evaluation
from cdlib.evaluation import normalized_mutual_information, variation_of_information, jaccard_index
import community as community_louvain
from tqdm import tqdm
from itertools import combinations
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.quality import modularity

try:
    import infomap
except:
    !pip install infomap
    import infomap

try:
    import clusim
except:
    !pip install clusim

plt.style.use('ggplot')
%matplotlib inline

REPORT_FOLDER = "report"
PLOTS_FOLDER = "plots"

CLUSTERS_FOLDER = "partitions_clu"

for folder in [REPORT_FOLDER, PLOTS_FOLDER, CLUSTERS_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)

for folder in [REPORT_FOLDER, PLOTS_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)


## 1. Utility Functions
Here we define helper functions used throughout the notebook.

In [ ]:
def load_network(path):
    """
    Load a Pajek-format network and convert to simple undirected NetworkX graph.

    Parameters:
        path (str): Path to .net file.

    Returns:
        G (networkx.Graph): Cleaned graph with integer-labeled nodes.
    """
    G = nx.read_pajek(path)
    G = nx.Graph(G)
    G = nx.convert_node_labels_to_integers(G)
    return G

def detect_communities_infomap(G):
    """
    Apply Infomap algorithm to detect communities in G.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list[set]): Detected communities.
    """
    im = infomap.Infomap()
    mapping = im.add_networkx_graph(G)
    im.run()
    modules = im.get_modules()
    community_dict = defaultdict(set)
    for node, com in modules.items():
        community_dict[com].add(mapping[node])
    return list(community_dict.values())


def detect_communities_louvain(G):
    """
    Apply Louvain algorithm to detect communities in G using NetworkX interface.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list[set]): Detected communities.
    """
    return list(nx.community.louvain_communities(G))


def detect_communities_greedy(G):
    """
    Apply Greedy modularity algorithm to detect communities in G.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list[set]): Detected communities.
    """
    return list(greedy_modularity_communities(G))

def ground_truth_partition(N=300, block_size=60):
    """
    Generate the ground-truth partition from known SBM structure.

    Parameters:
        N (int): Total number of nodes.
        block_size (int): Size of each block.

    Returns:
        list[set]: Ground truth communities.
    """
    return [set(range(i * block_size, (i + 1) * block_size)) for i in range(N // block_size)]

def save_partition_to_clu(partition, path, N=300):
    """
    Save a partition (list of sets) to a Pajek .clu file.

    Parameters:
        partition (list of sets): Detected communities.
        path (str): Output path to save the .clu file.
        N (int): Total number of nodes in the graph.
    """
    node_labels = [0] * N
    for i, community in enumerate(partition):
        for node in community:
            node_labels[node] = i + 1

    with open(path, 'w') as f:
        f.write(f"*Vertices {N}\n")
        for label in node_labels:
            f.write(f"{label}\n")


## 2. Network Loading and Preprocessing

### 2.1 Load Networks
We load all synthetic networks from the folder `data/`, generated according to the Stochastic Block Model. Each network has 300 nodes and 5 blocks. The only parameter that changes is `prr`, ranging from 0.02 to 1.00.

Each filename has the form:
`synthetic_network_N_300_blocks_5_prr_{value}_prs_0.02.net`

In [ ]:

from glob import glob
import re

networks = {}

for path in sorted(glob("data/*.net")):
    match = re.search(r"prr_([0-9.]+)_prs", path)
    if match:
        prr = float(match.group(1))
        networks[prr] = load_network(path)

networks = dict(sorted(networks.items()))  # Sort by prr




## 3. Community Detection Analysis
In this section, we apply the three required community detection algorithms to each network and store the partitions for further comparison.

In [ ]:
infomap_partitions = {}
louvain_partitions = {}
greedy_partitions = {}

with tqdm(total=len(networks), desc="Processing networks") as pbar:
    for prr, G in networks.items():
        infomap_partitions[prr] = detect_communities_infomap(G)
        louvain_partitions[prr] = detect_communities_louvain(G)
        greedy_partitions[prr] = detect_communities_greedy(G)
        pbar.update(1)
        pbar.set_postfix({"Current prr": f"{prr:.2f}"})

for algo_name, partitions in zip(
    ["infomap", "louvain", "greedy"],
    [infomap_partitions, louvain_partitions, greedy_partitions]
):
    for prr, partition in partitions.items():
        prr_str = f"{prr:.2f}".replace(".", "_")
        filename = f"{algo_name}_prr_{prr_str}.clu"
        path = os.path.join(CLUSTERS_FOLDER, filename)
        save_partition_to_clu(partition, path)

## 4. Comparison with Ground Truth

### 4.1 Compute Jaccard Index, NMI, and NVI
Here, we compare the community partitions obtained with each algorithm to the ground truth using standard metrics: Jaccard Index, Normalized Mutual Information (NMI), and Normalized Variation of Information (NVI).

In [ ]:
from cdlib import NodeClustering

results = []

gt = ground_truth_partition()

for prr in networks:
    for algo_name, partitions in zip(
        ["Infomap", "Louvain", "Greedy"],
        [infomap_partitions, louvain_partitions, greedy_partitions]
    ):
        part = partitions[prr]

        gt_nc = NodeClustering(communities=gt, graph=None)
        part_nc = NodeClustering(communities=part, graph=None)

        jacc = jaccard_index(gt_nc, part_nc).score
        nmi = normalized_mutual_information(gt_nc, part_nc).score
        nvi = variation_of_information(gt_nc, part_nc).score

        results.append({
            "prr": prr,
            "algorithm": algo_name,
            "Jaccard": jacc,
            "NMI": nmi,
            "NVI": nvi
        })

df_scores = pd.DataFrame(results)
df_scores.to_csv(os.path.join(REPORT_FOLDER, "metrics_scores.csv"), index=False)
df_scores


### 4.2 Plots
We analyze how the number of detected communities and modularity change as a function of `prr`.

#### 4.2.1 Number of Detected Communities
We count the number of communities detected by each algorithm for every value of `prr`, and plot the evolution to understand how the resolution of the detected structure changes.

In [ ]:
COLOR_PALETTE = {
    'Infomap': '#FF0000',
    'Louvain': '#0000FF',
    'Greedy': '#00FF00'
}

num_communities = []

for prr in networks:
    for algo_name, partitions in zip(
        ["Infomap", "Louvain", "Greedy"],
        [infomap_partitions, louvain_partitions, greedy_partitions]
    ):
        part = partitions[prr]
        num_communities.append({
            "prr": prr,
            "algorithm": algo_name,
            "num_communities": len(part)
        })

num_df = pd.DataFrame(num_communities)
num_df.to_csv(os.path.join(REPORT_FOLDER, "communities_count.csv"), index=False)

plt.figure(figsize=(8, 5))
for algo in num_df['algorithm'].unique():
    subset = num_df[num_df['algorithm'] == algo]
    plt.plot(subset['prr'],
             subset['num_communities'],
             label=algo,
             color=COLOR_PALETTE[algo],
             linewidth=2.5)
plt.xlabel("prr")
plt.ylabel("Number of communities")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_FOLDER, "num_communities_vs_prr.png"))
plt.show()


#### 4.2.2 Modularity vs prr
We compute the modularity of each partition for each algorithm and plot its evolution as `prr` increases, to evaluate the strength of the detected community structure.

In [ ]:

modularity_data = []

for prr, G in networks.items():
    for algo_name, partitions in zip(
        ["Infomap", "Louvain", "Greedy"],
        [infomap_partitions, louvain_partitions, greedy_partitions]
    ):
        part = partitions[prr]
        mod = modularity(G, part)
        modularity_data.append({
            "prr": prr,
            "algorithm": algo_name,
            "modularity": mod
        })

mod_df = pd.DataFrame(modularity_data)
mod_df.to_csv(os.path.join(REPORT_FOLDER, "modularity_scores.csv"), index=False)

plt.figure(figsize=(8, 5))
for algo in mod_df['algorithm'].unique():
    subset = mod_df[mod_df['algorithm'] == algo]
    plt.plot(subset['prr'],
             subset['modularity'],
             label=algo,
             color=COLOR_PALETTE[algo],
             linewidth=2.5)
plt.xlabel("prr")
plt.ylabel("Modularity")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_FOLDER, "modularity_vs_prr.png"))
plt.show()



##### 4.2.3 Jaccard Index vs prr
This plot shows how the Jaccard index evolves as `prr` increases. Since `prr` represents the probability of intra-community connections, increasing it should reinforce the community structure. Therefore, we expect the Jaccard index, which measures similarity to the ground truth, to increase as `prr` increases.

In [ ]:
plt.figure(figsize=(8, 5))
for algo in df_scores["algorithm"].unique():
    subset = df_scores[df_scores["algorithm"] == algo]
    plt.plot(subset['prr'],
             subset['Jaccard'],
             label=algo,
             color=COLOR_PALETTE[algo],
             linewidth=2.5)
plt.xlabel("prr")
plt.ylabel("Jaccard")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_FOLDER, "jaccard_vs_prr.png"))
plt.show()

#### 4.2.4 Normalized Mutual Information (NMI) vs prr
This plot shows the behavior of NMI, a similarity-based index. Just like Jaccard, we expect NMI to increase with `prr`, as community structure becomes more clearly defined.

In [ ]:
plt.figure(figsize=(8, 5))
for algo in df_scores["algorithm"].unique():
    subset = df_scores[df_scores["algorithm"] == algo]
    plt.plot(subset['prr'],
             subset['NMI'],
             label=algo,
             color=COLOR_PALETTE[algo],
             linewidth=2.5)
plt.xlabel("prr")
plt.ylabel("NMI")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_FOLDER, "nmi_vs_prr.png"))
plt.show()

#### 4.2.5 Variation of Information (NVI) vs prr
NVI is a distance metric: the higher it is, the more different the detected partition is from the ground truth. Since increasing `prr` reinforces community structure, the detected partitions should be closer to the ground truth. Thus, we expect NVI to decrease with `prr`.

In [ ]:
plt.figure(figsize=(8, 5))
for algo in df_scores["algorithm"].unique():
    subset = df_scores[df_scores["algorithm"] == algo]
    plt.plot(subset['prr'],
             subset['NVI'],
             label=algo,
             color=COLOR_PALETTE[algo],
             linewidth=2.5)
plt.xlabel("prr")
plt.ylabel("NVI")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_FOLDER, "nvi_vs_prr.png"))
plt.show()

## 5. Network Visualization

### 5.1 Color-coded Community Visualization for prr = 0.02, 0.16, 1.00
We visualize the community structure detected by each algorithm at three different values of `prr`.

In [ ]:

reference_G = networks[1.00]
positions = nx.spring_layout(reference_G, seed=42)

def community_colors(partition, G):
    color_map = {}
    for i, community in enumerate(partition):
        for node in community:
            color_map[node] = i
    return [color_map[node] for node in G.nodes()]

selected_prr = [0.02, 0.16, 1.00]
algorithms = [
    ("Infomap", infomap_partitions),
    ("Louvain", louvain_partitions),
    ("Greedy", greedy_partitions)
]

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 12))

for i, (algo_name, partition_dict) in enumerate(algorithms):
    for j, prr in enumerate(selected_prr):
        G = networks[prr]
        partition = partition_dict[prr]
        color_list = community_colors(partition, G)

        ax = axes[i, j]
        nx.draw_networkx_nodes(G, pos=positions, node_color=color_list,
                               cmap=plt.cm.tab20, node_size=20, ax=ax)
        nx.draw_networkx_edges(G, pos=positions, alpha=0.3, ax=ax)
        ax.set_title(f"{algo_name} - prr={prr:.2f}")
        ax.set_axis_off()

plt.tight_layout()
plt.savefig(os.path.join(PLOTS_FOLDER, "community_grid_visualization.png"))
plt.show()


## 6. Discussion

### 6.1 Limitations of Modularity
Below we address some conceptual questions to guide the discussion on modularity as an indicator of community structure.

**Q1: What are the limitations of modularity as a metric for evaluating community detection results?**

Modularity-based algorithms (Louvain, Greedy) suffer from the **resolution limit**: in large networks, small but well-formed communities may be merged into larger ones if doing so increases the overall modularity. As a result, meaningful small-scale structures can be missed.

Additionally, modularity may be **misleading in random networks**: even networks generated without any community structure (Erdős-Rényi) can exhibit relatively high modularity values due to random fluctuations. This makes it dangerous to rely solely on Q to infer the presence of meaningful communities.

**Q2: Can we state that a network with Q=0.4 has community structure? Why?**

Based on the observations previously made, although a modularity value of 0.4 might indicate a certain degree of community structure, this alone is not a reliable indicator.



### 6.2 Observations on Differences between Algorithms
This section provides a set of discussion questions aimed at analyzing the behavior of the three algorithms across different levels of noise (`prr`).

**Q1: What is the difference between Louvain, Greedy and Infomap Algorithms?**

MLouvain and Greedy are both modularity-based algorithms, but they differ in their strategy.

**Louvain** is a hierarchical algorithm that works in two main steps. First, it performs local modularity optimization by moving individual nodes into neighboring communities if it increases modularity. Then, it aggregates nodes belonging to the same community into a supernode and repeats the process on this new, compressed graph.

**Greedy**, is an agglomerative approach. It starts with each node in its own community and iteratively merges the pair of communities that leads to the highest increase in modularity. This process continues until all nodes are merged into a single community, and the division with the best modularity score is selected.

**Infomap**, in contrast, is not based on modularity. It takes a dynamic perspective: it simulates a random walker moving across the network and finds the partition that minimizes the expected description length of its trajectory.


**Q2: Are the modularity values obtained equal? Why?**

No, the modularity values obtained by the three algorithms are not always equal, and this is expected due to fundamental differences in how they operate.
  - Louvain and Greedy explicitly optimize modularity, so they generally produce higher modularity values. As prr increases, indicating stronger and more detectable community structure, the modularity increase
  - Infomap, on the other hand, is not based on modularity but on minimizing the description length of a random walk. As a result, the modularity of its partitions is not necessarily high and can even be unstable for low values of prr. This is visible in the plot, where infomap shows low modularity where prr is between 0.0 and 0.2, suggesting that its partitioning at that range doeas not align with what modularity considers optimal.